In [126]:
var('m n k')
R = QQ[m, k][n]

In [127]:
def h_list(f, g):    
    F, G = factor(f), factor(g)
    
    H = []
    
    for (u, m_u) in list(F):
        for (v, m_v) in list(G):
            ud, vd = u.degree(), v.degree()
            if ud == vd:
                A, B = u[ud-1]/(u[ud] * ud), v[vd-1]/(v[vd] * vd)
                h = A - B
                if h.denominator() == 1 and h.numerator() >= 0 and v(n) == u(n-h):
                    H.append(h.numerator())
    
    return sorted(H)

In [234]:
def reduce_poly(Z, f, g):
    p, q = deepcopy(f), deepcopy(g)
    c = 1
    
    for h in h_list(f, g):
        s = gcd(p(n = n), q(n = n+h))
        p = p/s 
        q = q/s(n = n-h)
        
        c *= prod([s(n = n-j) for j in range(1, int(h)+1)])
    
    return (R(Z * p), R(q), R(c))

In [235]:
def deduce_x(a, b, c, d):
    if not d and d != 0:
        return False

    xs = [var("x_" + str(i)) for i in range(int(d)+1)]
    
    x = sum([xs[i] * n^i for i in range(int(d)+1)])

    equation = (a(n=n) * x(n=n+1) - b(n=n-1) * x(n=n) - c(n=n)).full_simplify().coefficients(n)
    
    solution = solve([eqn[0] == 0 for eqn in equation], xs)
    
    return sum([list(solution[i].iterator())[1]*(n^i) for i in range(d+1)]) \
                    if solution else False

In [236]:
def guess_x_length(a, b, c):
    ad, bd, cd = a.degree(), b.degree(), c.degree()
    
    if (ad != bd) or (a[ad] != b[bd]):
        return cd - max(ad, bd)
    else:
        if a[ad-1] != b[bd-1]:
            return cd - ad + 1
        else:
            d = a[ad-1]/a[ad] - b[bd-1]/b[bd]
            if d.denominator() == 1 and d.numerator() >= 0:
                return d.numerator()
            else:
                return False

In [239]:
def gosper_sum(t):    
    r = (t(k=n+1)/t(k=n)).full_simplify()
    
    f_, g_ = R(r.numerator()), R(r.denominator())
    Z = f_[f_.degree()]/g_[g_.degree()]
    f, g = R(f_ / f_[f_.degree()]), R(g_ / g_[g_.degree()])
    
    a, b, c = reduce_poly(Z, f, g)
    d = guess_x_length(a, b, c)
    
    x = deduce_x(a, b, c, d)
    
    return b(n=n-1) * x(n=n) / c(n=n) * t(k=n) if x else False

In [248]:
def WZ(A, B):
    F = (A/B).full_simplify()
    dF = F(m=m+1) - F(m=m)
    
    return gosper_sum(dF)